### FINAL PROJECT - JEN, THEA, SARATH 
### --- DATING ANALYSIS ---
### PREDICT WHETHER DATERS WANT TO MEET AGAIN

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as pltt
import seaborn as sns
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer, fbeta_score
scorer = make_scorer(fbeta_score, beta=1.5)

# Some warnings tend to pop up during grid search
import warnings
warnings.filterwarnings('ignore')

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [27]:
data = pd.read_csv('Data/Speed_Dating_Data.csv', encoding='latin-1', engine="python")

print(data.shape[0], 'observations and', data.shape[1], 'features')

8378 observations and 195 features


In [28]:
# What kind of data are we dealing with?
data.dtypes.value_counts()

float64    174
int64       13
object       8
dtype: int64

In [29]:
# Men (1) and women (0)
data.groupby('iid').mean().gender.value_counts()

1.0    277
0.0    274
Name: gender, dtype: int64

In [30]:
# What percentage of pairs resulted in a match?
data['match'].value_counts(normalize=True)

0    0.835283
1    0.164717
Name: match, dtype: float64

In [31]:
data.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


In [32]:
# Waves 6-11 rated importance, others allocated 100 points
points = (data.wave != 6 ) &\
    (data.wave != 7 ) &\
    (data.wave != 8 ) &\
    (data.wave != 9 ) &\
    (data.wave != 10 ) &\
    (data.wave != 11 )

data = data[points]

In [33]:
match_corrs = data.select_dtypes(include=[np.number])\
.corrwith(data.match)\
.sort_values(ascending=False)

match_corrs = match_corrs[match_corrs > .25].index

data = data.dropna(subset=['id', 'pid'], axis=0)

for i in match_corrs[1:]:
    del data[i]

# Other columns that are too predictive
del data['int_corr']
del data['them_cal']
del data['you_call']

del data['field'] # redundant

match_corrs

Index(['match', 'dec_o', 'dec', 'like_o', 'like', 'fun_o', 'fun', 'shar_o',
       'shar', 'attr_o', 'attr', 'prob_o', 'prob'],
      dtype='object')

In [34]:
def fix_numeric(col, df=data, median=False):
    try:
        df[col] = df[col].str.replace(',', '')
    except:
        print('Column is not a string!')
    
    df[col] = df[col].astype(float)
    
    if median:
        df[col] = df[col].fillna(data.fillna.median())
    
    else:
        df[col] = df[col].fillna(-100)
    
    return df

for i in 'zipcode mn_sat tuition income'.split():
    data = fix_numeric(i)

In [35]:
for i in data.columns:
    if data[i].dtype == "O":
        data[i] = data[i].str.lower()
        print(i, end=', ')

undergra, from, career, 

In [36]:
from sklearn.dummy import DummyClassifier

data_benchmark = pd.get_dummies(data)
data_benchmark.fillna(data_benchmark.median(), inplace=True)

xbench = data_benchmark.drop('match', axis=1)
ybench = data_benchmark['match']

dum = DummyClassifier(strategy='constant', constant=1)

cvs = cross_val_score(dum, xbench, ybench, scoring=scorer, cv=5)

benchmark_score = cvs.mean()

print('Naive Benchmark:', benchmark_score)

Naive Benchmark: 0.39567670124395443


In [37]:
data['from_new_england'] = 0
for i in 'massachu connect rhode vermont vt hampsh maine boston cambridge'.split():
    data['from_new_england'] = np.where((data['from'].str.contains(i)) |\
                                   (data['undergra'].str.contains(i)), 1,
                                        data['from_new_england'])
    
data['from_china'] = 0
for i in 'china beijing shanghai hong taiwan'.split():
    data['from_china'] = np.where((data['from'].str.contains(i)) |\
                                  (data['undergra'].str.contains(i)), 1,
                                  data['from_china'])

data['from_india'] = 0
for i in 'india delhi bangalore'.split():
    data['from_india'] = np.where((data['from'].str.contains(i)) |\
                                  (data['undergra'].str.contains(i)), 1,
                                  data['from_india'])
    
data['from_europe'] = 0
for i in 'europe germany italy france spain poland portugal netherlands holland sweden switz greece belgium paris rome'.split():
    data['from_europe'] = np.where((data['from'].str.contains(i)) |\
                                   (data['undergra'].str.contains(i)), 1,
                                   data['from_europe'])

data['from_uk'] = 0
for i in 'london england uk britain scotland ireland kingdom oxford'.split():
    data['from_uk'] = np.where((data['from'].str.contains(i)) |\
                                   (data['undergra'].str.contains(i)), 1,
                               data['from_uk'])

data['from_ny'] = 0
for i in ['new york', 'ny']:
    data['from_ny'] = np.where((data['from'].str.contains(i)) |\
                               (data['undergra'].str.contains(i)), 1,
                               data['from_ny'])

data['from_nj'] = 0
for i in ['new jersey', 'nj']:
    data['from_nj'] = np.where((data['from'].str.contains(i)) |\
                               (data['undergra'].str.contains(i)), 1,
                               data['from_nj'])

data['from_california'] = 0
for i in 'cali diego francisco jose davis sacramento oakland clara angeles ucla stanford berkeley alto torrance'.split():
    data['from_california'] = np.where((data['from'].str.contains(i)) |\
                                   (data['undergra'].str.contains(i)), 1,
                                       data['from_california'])
    
data['from_texas'] = 0
for i in 'texas tx dallas austin houston dfw antonio'.split():
    data['from_texas'] = np.where((data['from'].str.contains(i)) |\
                                   (data['undergra'].str.contains(i)), 1,
                                  data['from_texas'])
    

In [38]:
data['career_education'] = 0
for i in 'professor teach academ educ'.split():
    data['career_education'] = np.where(data['career'].str.contains(i),
                                        1, data['career_education'])

data['career_law'] = 0
for i in 'law attorney legal defender counsel'.split():
    data['career_law'] = np.where(data['career'].str.contains(i),
                                  1, data['career_law'])

data['career_medicine'] = 0
for i in 'doctor dr physician md medical m.d. cardio dentist surg'.split():
    data['career_medicine'] = np.where(data['career'].str.contains(i),
                                       1, data['career_medicine'])

data['career_business'] = 0
for i in 'business mba m.b.a. consult manage ceo c.e.o. entre finance venture market strategy invest bank equity'.split():
    data['career_business'] = np.where(data['career'].str.contains(i),
                                       1, data['career_business'])
    
data['career_science'] = 0
for i in 'sci research biolo chemi'.split():
    data['career_science'] = np.where(data['career'].str.contains(i),
                                      1, data['career_science'])

data['career_gov'] = 0
for i in 'gov diplo poli'.split():
    data['career_gov'] = np.where(data['career'].str.contains(i),
                                  1, data['career_gov'])


In [39]:
data['school_columbia'] = 0
for i in 'columbia'.split():
    data['school_columbia'] = np.where(data['undergra'].str.contains(i),
                                       1, data['school_columbia'])

# Ivy league and other prestigious schools
data['school_ivy'] = 0
for i in 'dartmouth cornell princeton penn yale brown harvard stanford mit berkeley oxford'.split():
    data['school_ivy'] = np.where(data['undergra'].str.contains(i),
                                  1, data['school_ivy'])

In [40]:
# Dummify
data = pd.get_dummies(data, prefix='dummy')

# Some Stackoverflow code to fix duplicate column names
cols = pd.Series(data.columns)

for dup in data.columns.get_duplicates():
    cols[data.columns.get_loc(dup)] =\
    [dup+'.'+str(d_idx) if d_idx!=0 else dup for d_idx in range(data.columns.get_loc(dup).sum())]

data.columns = cols

data.shape

(5761, 820)

In [41]:
# Just a sample of how much missing data we have.
data.isnull().sum().sort_values(ascending=False)[:10]

num_in_3    5278
numdat_3    5067
shar4_3     4271
attr7_3     4271
sinc7_3     4271
intel7_3    4271
fun7_3      4271
amb7_3      4271
attr4_3     4271
sinc4_3     4271
dtype: int64

In [54]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
# Get null columns
null_cols = data.isnull().sum()
null_cols = list(null_cols[null_cols != 0].index)

for i in null_cols:
    print(i, 'has', data[i].isnull().sum(), 'missing values')
    x = data.fillna(data[i].mean()).drop(['match', 'id'], axis=1)
    y = x.pop(i)
    
    xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=.2)
    reg = KNeighborsRegressor()
    reg.fit(xtrain, ytrain)      
    pred = reg.predict(xtest)
    r2 = r2_score(ytest, pred)
    
    # If we can reasonably predict these values, do so
    if r2 > .2:
        print('regressing', i, 'with an r2 of', round(r2, 3))
        data['predicted'] = reg.predict(data.fillna(data[i].median()).drop([i, 'match', 'id'], axis=1))
        data[i] = np.where(data[i].isnull(), data['predicted'], data[i])
        del data['predicted']
    
    # Otherwise, just take the median
    else:
        print('averaging', i)
        data[i] = data[i].fillna(data[i].median())
    print()

intel3_1 has 100 missing values
regressing intel3_1 with an r2 of 0.998

amb3_1 has 100 missing values
regressing amb3_1 with an r2 of 0.998

attr5_1 has 1900 missing values
regressing attr5_1 with an r2 of 0.997

sinc5_1 has 1900 missing values
regressing sinc5_1 with an r2 of 0.981

intel5_1 has 1900 missing values
regressing intel5_1 with an r2 of 0.994

fun5_1 has 1900 missing values
regressing fun5_1 with an r2 of 0.993

amb5_1 has 1900 missing values
regressing amb5_1 with an r2 of 0.995

sinc has 232 missing values
regressing sinc with an r2 of 0.219

intel has 242 missing values
regressing intel with an r2 of 0.285

amb has 505 missing values
regressing amb with an r2 of 0.26

met has 314 missing values
regressing met with an r2 of 0.898

match_es has 538 missing values
regressing match_es with an r2 of 0.988

attr1_s has 4134 missing values
regressing attr1_s with an r2 of 0.995

sinc1_s has 4134 missing values
regressing sinc1_s with an r2 of 0.99

intel1_s has 4134 missing v

In [55]:
def get_partner_data(pid, col):
    '''Looks up the person's partner and adds their data
    as new features. If the partner ID doesn't exist,
    returns a -1.'''
    try:
        partner = data[data['iid'] == pid].head(1)[col].iloc[0]
        if partner:
            return partner
        else:
            return -1
    except:
        return -1

# Income (where income data is available, take the log difference)
data['partner_income'] = data['pid'].apply(get_partner_data, col='income')
data['income_difference'] = np.where((data.partner_income == -1) |\
                                     (data.income == -1),
                                     -1, np.log1p(np.abs(data.income - data.partner_income)))

# Age
data['age'].fillna(data['age'].median(), inplace=True)
data['age_o'].fillna(data['age_o'].median(), inplace=True)
data['age_difference'] = data['age'] - data['age_o']